<a href="https://colab.research.google.com/github/Nery666/Nery666/blob/main/%D0%9F%D1%80%D0%B8%D0%BC%D0%B5%D1%80_%D0%BF%D1%80%D0%BE%D1%81%D1%82%D0%BE%D0%B9_%D0%B1%D0%B0%D0%B7%D0%BE%D0%B2%D0%BE%D0%B9_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from copy import deepcopy

# Пример простой базовой модели (упрощение LLM)
class BaseModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10, 1)

    def forward(self, x):
        return self.linear(x)

# LoRA адаптер (низкоранговая адаптация)
class LoRAAdapter(nn.Module):
    def __init__(self, base_dim=10, rank=4):
        super().__init__()
        self.A = nn.Linear(base_dim, rank, bias=False)
        self.B = nn.Linear(rank, base_dim, bias=False)

    def forward(self, x):
        return self.B(self.A(x))

# Модель с LoRA
class ModelWithLoRA(nn.Module):
    def __init__(self):
        super().__init__()
        self.base = BaseModel()
        self.lora = LoRAAdapter()

    def forward(self, x):
        base_out = self.base(x)
        lora_out = self.lora(x)
        return base_out + lora_out

# Функция одной внутренней адаптации (один шаг градиентного спуска)
def inner_adaptation(model, loss_fn, x, y, inner_lr):
    adapted_model = deepcopy(model)
    optimizer = optim.SGD(adapted_model.lora.parameters(), lr=inner_lr)
    optimizer.zero_grad()
    pred = adapted_model(x)
    loss = loss_fn(pred, y)
    loss.backward()
    optimizer.step()
    return adapted_model

# MAML обучение
def maml_training_step(model, loss_fn, tasks, inner_lr, meta_lr):
    meta_optimizer = optim.Adam(model.parameters(), lr=meta_lr)
    meta_optimizer.zero_grad()
    meta_loss = 0.0

    for x_train, y_train, x_val, y_val in tasks:
        # Внутреннее обучение на одной задаче
        adapted_model = inner_adaptation(model, loss_fn, x_train, y_train, inner_lr)
        # Вычисляем ошибку на валидации после адаптации
        val_pred = adapted_model(x_val)
        val_loss = loss_fn(val_pred, y_val)
        meta_loss += val_loss

    meta_loss /= len(tasks)
    meta_loss.backward()
    meta_optimizer.step()

    return meta_loss.item()

# Демонстрация
if __name__ == "__main__":
    model = ModelWithLoRA()
    loss_fn = nn.MSELoss()

    # Сгенерируем примеры задач (по 4 задачи)
    tasks = []
    for _ in range(4):
        x_train = torch.randn(16, 10)
        y_train = torch.randn(16, 1)
        x_val = torch.randn(16, 10)
        y_val = torch.randn(16, 1)
        tasks.append((x_train, y_train, x_val, y_val))

    for step in range(50):
        loss = maml_training_step(model, loss_fn, tasks, inner_lr=0.01, meta_lr=0.001)
        if step % 10 == 0:
            print(f"Step {step}, Meta Loss: {loss:.4f}")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 1])) that is different to the input size (torch.Size([16, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Step 0, Meta Loss: 1.4526
Step 10, Meta Loss: 1.4526
Step 20, Meta Loss: 1.4526
Step 30, Meta Loss: 1.4526
Step 40, Meta Loss: 1.4526
